In [2]:
import numpy as np
import pandas as pd
import time
import heapq
import json
import psycopg2
import psycopg2.extras
import warnings
import sys
import uuid
import cvxpy as cp
import datetime
import argparse
from scipy.sparse import csr_matrix
from pandasql import sqldf

INF = float('inf')
total_time = time.time()


threadLimit = 100
queryID = 'EqrckJKW1FKFTrsLMrgZKX'
debug = 1

if queryID is None:
    print("No valid query id!")
    exit(1)

In [3]:
def ReportStatus(msg, flag, queryID, query_version, output=None):
    """
    Print message and update status in biz_model.biz_fir_query_parameter_definition.
    """
    sql = "update fll_t_dw.biz_fir_query_parameter_definition set python_info_data='{0}', success_flag='{1}', update_time='{2}', python_result_json='{3}', version={5} where id='{4}'".format(msg, flag, datetime.datetime.now(), output, queryID, query_version)
    print("============================================================================================================================")
    print("Reporting issue:", msg)
    conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
    conn.autocommit = True
    cur = conn.cursor()
    cur.execute(sql)
    sql = "update fll_t_dw.biz_fir_third_parameter_definition set python_info_data='{0}', success_flag='{1}', update_time='{2}', python_result_json='{3}', version={5} where id='{4}'".format(msg, flag, datetime.datetime.now(), output, queryID, query_version)
    cur.execute(sql)
    conn.commit()
    conn.close()

def ConnectDatabase(queryID):
    """
    Load parameters in JSON from biz_model.biz_fir_query_parameter_definition and load data from biz_model.biz_ads_fir_pkg_data.
    """
    try:
        print('Parameters reading...')
        sqlParameter = "select query_id, query_based_version, query_version, python_json from fll_t_dw.biz_fir_third_parameter_definition where id='{0}'".format(queryID)
        conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
        paramInput = pd.read_sql(sqlParameter, conn)
        if paramInput.shape[0] == 0:
            raise Exception("No Valid Query Request is Found!")
        elif paramInput.shape[0] > 1:
            raise Exception("More than One Valid Query Requests are Found!")
        param = json.loads(paramInput['python_json'][0])
        data_query_id = paramInput['query_id'][0]
        query_based_verion = paramInput['query_based_version'][0]
        query_version = paramInput['query_version'][0]
        print("base version: {0}\ncurrent version: {1}\nobjective: {2}".format(query_based_verion, query_version, param["objective"]))
    except Exception as e:
        print("Loading Parameters from GreenPlum Failed!\n", e)
        exit(1)
    try:
        print('Data loading...')
        sqlInput = \
        """
        select billing_status_fz as billing, unit_id_fz as unit_id, product, fleet_year_fz as fleet_year, contract_cust_id as customer, contract_num,
        contract_lease_type as contract, cost, nbv, age_x_ceu as weighted_age, ceu_fz as ceu, teu_fz as teu, rent, rml_x_ceu_c as rml, cust_country, per_diem_rate_current as pdr
        from fll_t_dw.biz_ads_fir_pkg_data
        where query_id = '{1}'
        and unit_id_fz in
        (select unit_id
        from fll_t_dw.biz_fir_asset_package
        where query_version = {0}
        and query_id = '{1}')
        """.format(query_based_verion, data_query_id)
        data = pd.read_sql(sqlInput, conn)    
        if data.shape[0] == 0:
            raise Exception("No Data Available in GreenPlum!")
        print('Raw data shape:', data.shape)
        conn.close()
    except Exception as e:
        print(e)
        ReportStatus("Loading Data from GreenPlum Failed!", 'F', queryID, query_version)
        exit(1)
    return param, data, data_query_id, query_version

def OutputPackage(data, result, queryID, query_version):
    """
    Output final package to biz_model.biz_fir_asset_package.
    """
    sqlOutput = "insert into fll_t_dw.biz_fir_asset_package (unit_id, query_id, id, is_void, version, query_version) values %s"
    try:
        conn = psycopg2.connect(host = "10.18.35.245", port = "5432", dbname = "iflorensgp", user = "fluser", password = "13$vHU7e")
        conn.autocommit = True
        cur = conn.cursor()
        print('Writing data...')
        values_list = []
        for i in range(len(result)):
            if result[i]:
                values_list.append((data['unit_id'][i], queryID, uuid.uuid1().hex, 0, 0, int(query_version)))
        psycopg2.extras.execute_values(cur, sqlOutput, values_list)
        conn.commit()
        conn.close()
    except Exception as e:
        print(e) 
        ReportStatus("Writing data to GreenPlum Failed!", 'F', queryID, query_version)
        exit(1)

def DataProcessing(data):
    print("==============================================================")
    print('Data processing...')
    start_time = time.time()
    numData = data.shape[0]

    # Billing Status
    col = [i for i in range(numData)]
    row = [0 if data['billing'][i]=='ON' else 1 if data['billing'][i]=='OF' else 2 for i in range(numData)]
    statusOneHot = csr_matrix(([1 for _ in range(numData)], (row, col)), shape=(3, numData))

    # One hot all lessees
    lesseeIndex = {k: v for v, k in enumerate(data['customer'].value_counts().index)}
    row = [lesseeIndex[data['customer'][i]] for i in range(numData)]
    lesseeOneHot = csr_matrix(([1 for _ in range(numData)], (row, col)), shape=(len(data['customer'].value_counts()), numData))

    # One hot all contract number
    row = []
    contractIndex = {k: v for v, k in enumerate(data['contract_num'].value_counts().index)}
    row = [contractIndex[data['contract_num'][i]] for i in range(numData)]
    contractOneHot = csr_matrix(([1 for _ in range(numData)], (row, col)), shape=(len(data['contract_num'].value_counts()), numData))

    # One hot all contract type
    contractTypeIndex = {k: v for v, k in enumerate(data['contract'].value_counts().index)}
    row = [contractTypeIndex[data['contract'][i]] for i in range(numData)]
    contractTypeOneHot = csr_matrix(([1 for _ in range(numData)], (row, col)), shape=(len(data['contract'].value_counts()), numData))

    # One hot all product type
    productIndex = {k: v for v, k in enumerate(data['product'].value_counts().index)}
    row = [productIndex[data['product'][i]] for i in range(numData)]
    productOneHot = csr_matrix(([1 for _ in range(numData)], (row, col)), shape=(len(data['product'].value_counts()), numData))

    # Container Age
    row, col = [], [] # note: container age ranges may overlap
    for i in range(numData):
        for j in range(len(param['containersAge']['list'])):
            if param['containersAge']['list'][j]['containersAgeFrom'] <= data['fleet_year'][i] <= param['containersAge']['list'][j]['containersAgeTo']:
                row.append(j)
                col.append(i)
    containerAgeOneHot = csr_matrix(([1 for _ in range(len(row))], (row, col)), shape=(len(param['containersAge']['list']), numData))

    # Weighted Age
    row, col = [], []
    for i in range(numData):
        for j in range(len(param['weightedAge']['list'])):
            if param['weightedAge']['list'][j]['weightedAgeFrom'] <= data['weighted_age'][i] <= param['weightedAge']['list'][j]['weightedAgeTo']:
                row.append(j)
                col.append(i)
    weightedAgeOneHot = csr_matrix(([1 for _ in range(len(row))], (row, col)), shape=(len(param['weightedAge']['list']), numData))

    # RML
    row, col = [], []
    for i in range(numData):
        for j in range(len(param['rml']['list'])):
            if param['rml']['list'][j]['rmlFrom'] <= data['rml'][i] <= INF:
                row.append(j)
                col.append(i)
    rmlOneHot = csr_matrix(([1 for _ in range(len(row))], (row, col)), shape=(len(param['rml']['list']), numData))

    # Country
    row, col = [], []
    for i in range(numData):
        for j in range(len(param['country']['list'])):
            if data['cust_country'][i] in param['country']['list'][j]['country']:
                row.append(j)
                col.append(i)
    countryOneHot = csr_matrix(([1 for _ in range(len(row))], (row, col)), shape=(len(param['country']['list']), numData))

    print('Time cost:', time.time() - start_time)
    return statusOneHot, \
        lesseeIndex, lesseeOneHot, \
        contractIndex, contractOneHot, \
        contractTypeIndex, contractTypeOneHot, \
        productIndex, productOneHot, \
        containerAgeOneHot, weightedAgeOneHot, rmlOneHot, countryOneHot

def BuildModel(EnableWeightedAge=False, lookupTable=None):
    print("==============================================================")
    print('Building Model...')
    start_time = time.time()
    x = cp.Variable(shape=data.shape[0], boolean=True)
    x.value = np.ones(data.shape[0])
    if EnableWeightedAge:
        y = cp.Variable(shape=lookupTable.shape[0], boolean=True)
        x = y @ lookupTable

    # constraints
    constraints = [cp.sum(x) >= 1]

    # objective
    if param['objective']['objective'] == 'containerAge':
        obj = (x @ data['fleet_year']) / (cp.sum(x))
    elif param['objective']['objective'] == 'weightedAge':
        obj = (x @ data['weighted_age']) / (x @ data['ceu'])
    elif param['objective']['objective'] == 'lessee':
        lesseeValue = lesseeOneHot[lesseeIndex[param['objective']['type'][0]]].toarray().ravel() * data[param['objective']['basis']]
        obj = (x @ lesseeValue) / (x @ data[param['objective']['basis']])
        if param['objective']['value']:
            constraints.append(x @ lesseeValue <= param['objective']['value'] / 100 * x @ data[param['objective']['basis']])
    elif param['objective']['objective'] == 'onHire':
        obj = (x @ (statusOneHot[0].toarray().ravel() * data[param['objective']['basis']])) / (x @ data[param['objective']['basis']])
        if param['objective']['value']:
            constraints.append(x @ (statusOneHot[0].toarray().ravel() * data[param['objective']['basis']]) \
                               <= param['objective']['value'] / 100 * x @ data[param['objective']['basis']])
    elif param['objective']['objective'] == 'product':
        productListIndex = [productIndex.get(p) for p in param['objective']['type'] if productIndex.get(p) is not None]
        obj = cp.sum(productOneHot[productListIndex].toarray() @ cp.multiply(x, data[param['objective']['basis']])) / (x @ data[param['objective']['basis']])
        if param['objective']['value']:
            constraints.append(cp.sum(productOneHot[productListIndex].toarray() @ cp.multiply(x, data[param['objective']['basis']])) \
                               <= param['objective']['value'] / 100 * x @ data[param['objective']['basis']])
    elif param['objective']['objective'] == 'lesseeCountry':
        row, col = [], []
        for i in range(data.shape[0]):
            if data['cust_country'][i] in param['objective']['type']:
                row.append(j)
                col.append(i)
        countryOneHot = csr_matrix(([1 for _ in range(len(row))], (row, col)), shape=(1, data.shape[0]))
        obj = (x @ (countryOneHot[0].toarray().ravel() * data[param['objective']['basis']])) / (x @ data[param['objective']['basis']])
    elif param['objective']['objective'] == 'totalRentDscr':
        obj = x @ data['rent']
    elif param['objective']['objective'] == 'remainingLeaseTenor':
        obj = (x @ data['rml']) / (x @ data['ceu'])
        if param['objective']['value']:
            constraints.append(x @ data['rml'] <= param['objective']['value'] * x @ data['ceu'])
    elif param['objective']['objective'] == 'factory':
        obj = (x @ (statusOneHot[2].toarray().ravel() * data[param['objective']['basis']])) / (x @ data[param['objective']['basis']])
    elif param['objective']['objective'] == 'coc':
        obj = (x @ (data['pdr'] * data[param['objective']['basis']])) / (x @ data[param['objective']['basis']])
    else:
        print('Objective is not expected: {0}. Set dummy objective.'.format(param['objective']['objective']))
        obj = 1
    
    if param['objective']['value']:
        objective = cp.Maximize(obj)
    else:
        objective = cp.Maximize(obj) if param['objective']['maxOrMin'] else cp.Minimize(obj)


    # NBV
    if param['totalNBVFrom']:
        print('Set NBV Lower Bound')
        constraints.append(x @ data['nbv'] >= param['totalNBVFrom'])
    if param['totalNBVTo']:
        print('Set NBV Upper Bound')
        constraints.append(x @ data['nbv'] <= param['totalNBVTo'])
    # Cost
    if param['totalCostFrom']:
        print('Set Cost Lower Bound')
        constraints.append(x @ data['cost'] >= param['totalCostFrom'])
    if param['totalCostTo']:
        print('Set Cost Upper Bound')
        constraints.append(x @ data['cost'] <= param['totalCostTo'])
    # Rent
    if param['totalRentFrom']:
        print('Set Rent Lower Bound')
        constraints.append(x @ data['rml'] >= param['totalRentFrom'])
    # Average Fleet Age
    if param['containersAge']['average']['averageContainersAge']:
        print('Set Average Container Age Limit')
        constraints.append(
            (1 if param['containersAge']['average']['symbol'] else -1) * (
            x @ data['fleet_year']
            - param['containersAge']['average']['averageContainersAge'] * cp.sum(x)
            ) >= 0)
    # Fleet Age
    for i in range(len(param['containersAge']['list'])):
        print(f'Set Container Age {i} Limit')
        constraints.append(
            (1 if param['containersAge']['list'][i]['symbol'] else -1) * (
            x @ (containerAgeOneHot[i].toarray().ravel() * data[param['containersAge']['basis']])
            - param['containersAge']['list'][i]['percent'] / 100 * (x @ data[param['containersAge']['basis']])
            ) >= 0)
    # Average Weighted Age
    if param['weightedAge']['average']['averageWeighedAge']:
        print('Set Average Weighted Age Limit')
        constraints.append(
            (1 if param['weightedAge']['average']['symbol'] else -1) * (
            x @ data['weighted_age']
            - param['weightedAge']['average']['averageWeighedAge'] * (x @ data['ceu'])
            ) >= 0)
    # Weighted Age
    if EnableWeightedAge:
        for i in range(len(param['weightedAge']['list'])):
            print(f'Set Weighted Age {i} Limit')
            constraints.append(
                (1 if param['weightedAge']['list'][i]['symbol'] else -1) * (
                x @ weightedAgeOneHot[i].toarray().ravel()
                - param['weightedAge']['list'][i]['percent'] / 100 * (x @ data['ceu'])
                ) >= 0)
    # RML
    for i in range(len(param['rml']['list'])):
        print(f'Set RML {i} Limit')
        constraints.append(
            (1 if param['rml']['list'][i]['symbol'] else -1) * (
            x @ (rmlOneHot[i].toarray().ravel() * data[param['rml']['basis']])
            - param['rml']['list'][i]['percent'] / 100 * (x @ data[param['rml']['basis']])
            ) >= 0)
    # Status
    for i in range(len(param['status']['list'])):
        print(f'Set Status {i} Limit')
        status = 0 if param['status']['list'][i]['statusType'] == 'ON' else 1 if param['status']['list'][i]['statusType'] == 'OF' else 2
        constraints.append(
            (1 if param['status']['list'][i]['symbol'] else -1) * (
            x @ (statusOneHot[status].toarray().ravel() * data[param['status']['basis']])
            - param['status']['list'][i]['percent'] / 100 * (x @ data[param['status']['basis']])
            ) >= 0)
    # Product Type
    for i in range(len(param['product']['list'])):
        print(f'Set Product Type {i} Limit')
        productListIndex = [productIndex.get(p) for p in param['product']['list'][i]['productType'] if productIndex.get(p) is not None]
        constraints.append(
            (1 if param['product']['list'][i]['symbol'] else -1) * (
            cp.sum(productOneHot[productListIndex].toarray() @ cp.multiply(x, data[param['product']['basis']]))
            - param['product']['list'][i]['percent'] / 100 * (x @ data[param['product']['basis']])
            ) >= 0)
    # Contract Type
    for i in range(len(param['contractType']['list'])):
        print(f'Set Contract Type {i} Limit')
        contractTypeListIndex = [contractTypeIndex.get(c) for c in param['contractType']['list'][i]['contractType'] if contractTypeIndex.get(c) is not None]
        constraints.append(
            (1 if param['contractType']['list'][i]['symbol'] else -1) * (
            cp.sum(contractTypeOneHot[contractTypeListIndex].toarray() @ cp.multiply(x, data[param['contractType']['basis']]))
            - param['contractType']['list'][i]['percent'] / 100 * (x @ data[param['contractType']['basis']])
            ) >= 0)
    # Country
    for i in range(len(param['country']['list'])):
        print(f'Set Country {i} Limit')
        constraints.append(
            (1 if param['country']['list'][i]['symbol'] else -1) * (
            x @ (countryOneHot[i].toarray().ravel() * data[param['country']['basis']])
            - param['country']['list'][i]['percent'] / 100 * (x @ data[param['country']['basis']])
            ) >= 0)
    # Certain Lessee
    for i in range(len(param['lessee']['list'])):
        print(f'Set Lessee {i} Limit')
        if param['lessee']['list'][i]['lessee'] not in lesseeIndex:
            print('\t{0} does not exist in dataset.'.format(param['lessee']['list'][i]['lessee']))
            continue
        constraints.append(
            (1 if param['lessee']['list'][i]['symbol'] else -1) * (
            x @ (lesseeOneHot[lesseeIndex[param['lessee']['list'][i]['lessee']]].toarray().ravel() * data[param['lessee']['basis']])
            - param['lessee']['list'][i]['percent'] / 100 * (x @ data[param['lessee']['basis']])
            ) >= 0)
    # Top Lessee
    for i in range(3):
        if param['lessee']['topLessee'][f'top{i+1}']['percent']:
            print(f'Set Top Lessee {i+1} Limit')
            constraints.append(
                cp.sum_largest(lesseeOneHot.toarray() @ cp.multiply(x, data[param['lessee']['basis']]), i + 1)
                - param['lessee']['topLessee'][f'top{i+1}']['percent'] / 100 * (x @ data[param['lessee']['basis']])
                <= 0)
    # Other Lessee -- only handle certain lessees
    if param['lessee']['others']['percent'] and param['lessee']['others']['lessee']:
        print('Set Other Lessees Limit')
        otherLesseeIndex = [lesseeIndex.get(l) for l in param['lessee']['others']['lessee'] if lesseeIndex.get(l) is not None]
        constraints.append(
            cp.sum_largest(lesseeOneHot[otherLesseeIndex].toarray() @ cp.multiply(x, data[param['lessee']['basis']]), 1)
            - param['lessee']['others']['percent'] / 100 * (x @ data[param['lessee']['basis']])
            <= 0)

    # Num Limit
    if not EnableWeightedAge:
        if param['numContractProductLimit']:
            print('Set Num Limit')
            contractProductType = [c.toarray().ravel()*p.toarray().ravel() for c in contractOneHot for p in productOneHot if sum(c.toarray().ravel()*p.toarray().ravel())>0]
            delta = cp.Variable(shape=len(contractProductType), boolean=True)
            for i in range(len(contractProductType)):
                constraints.append(x @ contractProductType[i] >= param['numContractProductLimit'] * delta[i])
                constraints.append(x @ contractProductType[i] <= 99999999 * delta[i])

    prob = cp.Problem(objective, constraints)
    print('Time Cost:', time.time() - start_time)
    
    return x, prob

def Validation(x, EnableWeightedAge=False):
    epsilon = 0.001
    passed = True
    print("==============================================================")
    print('Validating...')
    print(int(sum(x)), '/', len(x), 'containers are selected.')
    if sum(x) == 0:
        return False
    # objective
    objective = (x @ data['nbv']) if param['prefer']['nbvorCost'] else (x @ data['cost'])
    print('Objective: {0} = {1}'.format('nbv' if param['prefer']['nbvorCost'] else 'cost', objective))
    # NBV
    if param['totalNBVFrom']:
        p = x @ data['nbv'] >= param['totalNBVFrom'] - epsilon
        if not p: print('NBV Lower Bound Failed')
        passed = passed and p
    if param['totalNBVTo']:
        p = x @ data['nbv'] <= param['totalNBVTo'] + epsilon
        if not p: print('NBV Upper Bound Failed')
        passed = passed and p
    # Cost
    if param['totalCostFrom']:
        p = x @ data['cost'] >= param['totalCostFrom'] - epsilon
        if not p: print('Cost Lower Bound Failed')
        passed = passed and p
    if param['totalCostTo']:
        p = x @ data['cost'] <= param['totalCostTo'] + epsilon
        if not p: print('Cost Upper Bound Failed')
        passed = passed and p
    # Rent
    if param['totalRentFrom']:
        p = x @ data['rml'] >= param['totalRentFrom'] - epsilon
        if not p: print('Rent Lower Bound Failed')
        passed = passed and p
    # Average Fleet Age
    if param['containersAge']['average']['averageContainersAge']:
        p = (1 if param['containersAge']['average']['symbol'] else -1) * (
            x @ data['fleet_year']
            - param['containersAge']['average']['averageContainersAge'] * cp.sum(x)
            ) >= - epsilon
        if not p: print('Average Fleet Age Failed')
        passed = passed and p
    # Fleet Age
    for i in range(len(param['containersAge']['list'])):
        p = (1 if param['containersAge']['list'][i]['symbol'] else -1) * (
            x @ (containerAgeOneHot[i].toarray().ravel() * data[param['containersAge']['basis']])
            - param['containersAge']['list'][i]['percent'] / 100 * (x @ data[param['containersAge']['basis']])
            ) >= - epsilon
        if not p: print(f'Fleet Age Limit {i} Failed')
        passed = passed and p
    # Average Weighted Age
    if param['weightedAge']['average']['averageWeighedAge']:
        p = (1 if param['weightedAge']['average']['symbol'] else -1) * (
            x @ data['weighted_age']
            - param['weightedAge']['average']['averageWeighedAge'] * (x @ data['ceu'])
            ) >= - epsilon
        if not p: print('Average Weighted Age Failed')
        passed = passed and p
    # Weighted Age
    if EnableWeightedAge:
        for i in range(len(param['weightedAge']['list'])):
            p = (1 if param['weightedAge']['list'][i]['symbol'] else -1) * (
                x @ weightedAgeOneHot[i].toarray().ravel()
                - param['weightedAge']['list'][i]['percent'] / 100 * (x @ data['ceu'])
                ) >= - epsilon
            if not p: print(f'Weighted Age Limit {i} Failed')
            passed = passed and p
    # RML
    for i in range(len(param['rml']['list'])):
        p = (1 if param['rml']['list'][i]['symbol'] else -1) * (
            x @ (rmlOneHot[i].toarray().ravel() * data[param['rml']['basis']])
            - param['rml']['list'][i]['percent'] / 100 * (x @ data[param['rml']['basis']])
            ) >= - epsilon
        if not p: print(f'RML Limit {i} Failed')
        passed = passed and p
    # Status
    for i in range(len(param['status']['list'])):
        status = 0 if param['status']['list'][i]['statusType'] == 'ON' else 1 if param['status']['list'][i]['statusType'] == 'OF' else 2
        p = (1 if param['status']['list'][i]['symbol'] else -1) * (
            x @ (statusOneHot[status].toarray().ravel() * data[param['status']['basis']])
            - param['status']['list'][i]['percent'] / 100 * (x @ data[param['status']['basis']])
            ) >= - epsilon
        if not p: print(f'Status {i} Limit Failed')
        passed = passed and p
    # Product Type
    for i in range(len(param['product']['list'])):
        productListIndex = [productIndex.get(p) for p in param['product']['list'][i]['productType'] if productIndex.get(p) is not None]
        p = (1 if param['product']['list'][i]['symbol'] else -1) * (
            cp.sum(productOneHot[productListIndex].toarray() @ (x * data[param['product']['basis']]))
            - param['product']['list'][i]['percent'] / 100 * (x @ data[param['product']['basis']])
            ) >= - epsilon
        if not p: print(f'Product Type {i} Limit Failed')
        passed = passed and p
    # Contract Type
    for i in range(len(param['contractType']['list'])):
        contractTypeListIndex = [contractTypeIndex.get(c) for c in param['contractType']['list'][i]['contractType'] if contractTypeIndex.get(c) is not None]
        p = (1 if param['contractType']['list'][i]['symbol'] else -1) * (
            cp.sum(contractTypeOneHot[contractTypeListIndex].toarray() @ (x * data[param['contractType']['basis']]))
            - param['contractType']['list'][i]['percent'] / 100 * (x @ data[param['contractType']['basis']])
            ) >= - epsilon
        if not p: print(f'Contract Type {i} Limit Failed')
        passed = passed and p
    # Country
    for i in range(len(param['country']['list'])):
        p = (1 if param['country']['list'][i]['symbol'] else -1) * (
            x @ (countryOneHot[i].toarray().ravel() * data[param['country']['basis']])
            - param['country']['list'][i]['percent'] / 100 * (x @ data[param['country']['basis']])
            ) >= - epsilon
        if not p: print(f'Country {i} Limit Failed')
        passed = passed and p
    # Certain Lessee
    for i in range(len(param['lessee']['list'])):
        if param['lessee']['list'][i]['lessee'] not in lesseeIndex:
            print('\t{0} does not exist in dataset.'.format(param['lessee']['list'][i]['lessee']))
            continue
        p = (1 if param['lessee']['list'][i]['symbol'] else -1) * (
            x @ (lesseeOneHot[lesseeIndex[param['lessee']['list'][i]['lessee']]].toarray().ravel() * data[param['lessee']['basis']])
            - param['lessee']['list'][i]['percent'] / 100 * (x @ data[param['lessee']['basis']])
            ) >= - epsilon
        if not p: print(f'Lessee {i} Limit Failed')
        passed = passed and p
    # Top Lessee
    for i in range(3):
        if param['lessee']['topLessee'][f'top{i+1}']['percent']:
            p = sum(heapq.nlargest(i + 1, lesseeOneHot.toarray() @ (x * data[param['lessee']['basis']]))) \
                - param['lessee']['topLessee'][f'top{i+1}']['percent'] / 100 * (x @ data[param['lessee']['basis']]) \
                <= epsilon
            if not p: print(f'Top {i+1} Lessee Failed')
            passed = passed and p
    # Other Lessee -- only handle certain lessees
    if param['lessee']['others']['percent'] and param['lessee']['others']['lessee']:
        otherLesseeIndex = [lesseeIndex.get(l) for l in param['lessee']['others']['lessee'] if lesseeIndex.get(l) is not None]
        p = max(lesseeOneHot[otherLesseeIndex].toarray() @ (x * data[param['lessee']['basis']])) \
            - param['lessee']['others']['percent'] / 100 * (x @ data[param['lessee']['basis']]) \
            <= epsilon
        if not p: print('Other Lessees Failed')
        passed = passed and p
    # Num Limit
    if not EnableWeightedAge:
        if param['numContractProductLimit']:
            contractProductType = [c.toarray().ravel()*p.toarray().ravel() for c in contractOneHot for p in productOneHot]
            p = min([c @ x for c in contractProductType if c @ x > 0]) >= param['numContractProductLimit'] - 1
            if not p: print('Num Limit Failed')
            passed = passed and p
    return passed


In [4]:
param, data, data_query_id, query_version = ConnectDatabase(queryID)

Parameters reading...
base version: 2
current version: 3
objective: {'objective': 'weightedAge', 'type': None, 'maxOrMin': '0', 'value': None, 'basis': None}
Data loading...
Raw data shape: (37633, 16)


In [5]:
model_time = time.time()
statusOneHot, \
lesseeIndex, lesseeOneHot, \
contractIndex, contractOneHot, \
contractTypeIndex, contractTypeOneHot, \
productIndex, productOneHot, \
containerAgeOneHot, weightedAgeOneHot, rmlOneHot, countryOneHot \
= DataProcessing(data)
x, prob = BuildModel()
print("==============================================================")
print('Model solving...')
prob.solve(solver=cp.CBC, verbose=0, warm_start=True, qcp=1)
print("Status:", prob.status)
print('Time Cost', time.time() - model_time)

Data processing...
Time cost: 1.258840560913086
Building Model...
Set NBV Upper Bound
Set Cost Upper Bound
Set Average Weighted Age Limit
Set Num Limit
Time Cost: 0.8660271167755127
Model solving...
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  problem proven infeasible
Model Status:  problem proven infeasible
Model Status:  problem proven infeasible
Model Status:  relaxation infeasible
Model Status:  solution
Model Status:  relaxation infeasible
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  relaxation infeasible
Model Status:  solution
Model Status:  solution
Model Status:  relaxation infeasible
Model Status:  relaxation infeasible
Model Status:  relaxation infeasible
Model Status:  relaxation infeasible
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  solution
Model Status:  relaxation infeasibl

In [6]:
if prob.status == 'infeasible':
    ReportStatus('Problem Proven Infeasible! Please Modify Constaints.', 'I', queryID, query_version)
    exit(0)

x = np.where(abs(x.value-1) < 1e-3, 1, 0) # x == 1
passed = Validation(x)
if not passed:
    OutputPackage(data, x, data_query_id, query_version)
    ReportStatus('Constraints Cannot Be fulfilled! Please Modify Constaints Or Increase Running Timelimit.', 'N', queryID, query_version)

# Passed
if not param['weightedAge']['list']:
    # if no limit on weighted age
    OutputPackage(data, x, data_query_id, query_version)
    ReportStatus('Algorithm Succeeded!', 'O', queryID, query_version)


Validating...
100 / 37633 containers are selected.
Objective: nbv = 93089.474
Writing data...
Reporting issue: Algorithm Succeeded!
